In [ ]:
# Filtering out the warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing the required libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss

# <font color = blue> BANK LOAN ANALYSIS </font>
This case study aims to identify patterns which indicate if a client has difficulty paying their installments which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc. This will ensure that the consumers capable of repaying the loan are not rejected. Identification of such applicants using EDA is the aim of this case study.

##  Task 1: Data Reading

- ### Subtask 1.1: Read the Application Data.

Read the application data file provided and store it in a dataframe `application`.

In [ ]:
# Read the csv file using 'read_csv'. I have moved the file into my working directory. Using set_option to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
application = pd.read_csv("application_data.csv")
application.head()

In [ ]:
# Check the number of rows and columns in the dataframe
application.shape

In [ ]:
# Check each column datatypes as well as Null counts of the dataframe
application.info(verbose = True, null_counts = True)

In [ ]:
#check the percentage of null/missing values for each column rounded off to 3 digits
round((application.isnull().sum()/len(application.index))*100,3)

In [ ]:
# Check the statistical summary for the numeric columns (65 float + 41 Integer columns)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)
application.describe()

In [ ]:
#code that provides a list of categorical columns from the dataset (16 columns)
list(set(application.columns) - set(application.describe().columns))

##  Inferences from Reading Data
1. Occupation_type field has 31% Null values. We can find patterns with the field and whether it can be imputed using any other field in the dataset or not<br>
2. Most of the fields related to building/property information (i.e. AVG, Mode, MEDI) is having greater than 50% Null values. We can try to analyze if there are related to any specific loan type? We can also take a decision whether to drop these columns or keep them. In a common data analysis scenario, we should not consider such columns to derive specific insights over the entire dataset<br><br>

Looking at the statistical summary from describe function, we can infer the below points<br>
1. DAYS_EMPLOYED field has some outliers which should be removed/ignored to perform good analysis on that field. In real life, we should ask for clarification on the wrong data and get the data corrected by the client<br>
2. AMT_TOTAL_INCOME field has outliers which should be removed/ignored to perform good analysis on that field<br><br>

Other Inferences
1. Age should be calculated and age groups can be defined to better utilize DAYS_BIRTH field<br>
2. Some of the major datapoints are identified by observing the data<br>
3. Finding categorical data which will be crucial in slicing the dataset to derive insights

##  Task 2: Data Analysis and Cleaning

- ### Subtask 2.1: Draw pie-chart to understand various customer attributes

In [ ]:
#customer attributes and composition
fig = plt.figure(figsize=(6,4), dpi=1600)
#1 rows 2 columns

#first row, first column
ax1 = plt.subplot2grid((1,2),(0,0))
application.TARGET.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%",textprops={'fontsize': 6})
plt.ylabel('Target', fontsize=5)
plt.title('Pays on Time vs Defaulters', fontsize=7)

#first row second column
ax1 = plt.subplot2grid((1,2), (0,1))
application.CODE_GENDER.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%",textprops={'fontsize': 6})
plt.ylabel('Gender', fontsize=5)
plt.title('Gender Composition', fontsize=7)

plt.subplots_adjust(wspace=1.0)
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,4), dpi=1600)
#1 rows 2 columns

#first row, first column
ax1 = plt.subplot2grid((1,2),(0,0))
application.NAME_EDUCATION_TYPE.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%",textprops={'fontsize': 6})
plt.ylabel('Education Type', fontsize=6)
plt.title('Education Type of Customers', fontsize=7)

#first row second column
ax1 = plt.subplot2grid((1,2), (0,1))
application.NAME_INCOME_TYPE.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%",textprops={'fontsize': 6})
plt.ylabel('Income Type', fontsize=6)
plt.title('Income Type of Customers', fontsize=7)


plt.subplots_adjust(wspace=1.0)
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,4), dpi=1600)
#1 rows 2 columns

#first row, first column
ax1 = plt.subplot2grid((1,2), (0,0))
application.FLAG_OWN_CAR.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%",textprops={'fontsize': 6})
plt.ylabel('Own Car', fontsize=5)
plt.title('Car Ownership of Customers', fontsize=7)

#first row second column
ax1 = plt.subplot2grid((1,2), (0,1))
application.FLAG_OWN_REALTY.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%",textprops={'fontsize': 6})
plt.ylabel('Own House', fontsize=5)
plt.title('House Ownership of Customers', fontsize=7)

plt.subplots_adjust(wspace=1.0)
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,4), dpi=1600)
#1 rows 2 columns

#first row, first column
ax1 = plt.subplot2grid((1,2), (0,0))
application.NAME_FAMILY_STATUS.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%",textprops={'fontsize': 6})
plt.ylabel('Family Status', fontsize=5)
plt.title('Family Status of Customers', fontsize=7)

#first row second column
ax1 = plt.subplot2grid((1,2), (0,1))
application.REGION_RATING_CLIENT.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%",textprops={'fontsize': 6})
plt.ylabel('Region Rating', fontsize=5)
plt.title('Region Rating of Customers', fontsize=7)

plt.subplots_adjust(wspace=1.0)
plt.show()

- ### Subtask 2.2: Find major variables to be analyzed

In [ ]:
##some major variables to be analysed based on observing raw data. Ensure to use head() function, if not used, might face disk failures on slower machines
application[['SK_ID_CURR','TARGET','DAYS_BIRTH','DAYS_EMPLOYED','AMT_INCOME_TOTAL','AMT_CREDIT','NAME_CONTRACT_TYPE',
             'CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE',
             'NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','ORGANIZATION_TYPE','REGION_RATING_CLIENT']].head(100)

- ### Subtask 2.3: create AGE field, AGE_GROUP categorical field and derive insights based on age group

In [ ]:
#calculating age based on number of days_birth and converting into year. Each year is calculated as 365 days ignoring effect of leap year 
application['AGE'] = application['DAYS_BIRTH'].div(365).round(1).abs()

In [ ]:
#Converting AGE into categorical data to draw better insights based on age group
application["AGE_GROUP"] = pd.cut(application.AGE,[0,30,45,60,9999],labels=["<30","30-45","45-60","60+"])
application.AGE_GROUP.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%")
plt.title('Customer Composition based on Age Group', fontsize = 12)
plt.ylabel("Age Group", fontsize=10)
plt.show()

In [ ]:
#understanding the default amounts per age group.
#First groupby the required fields and sumup the total amount of loan per target and age_group
pd.set_option('display.float_format', '{:.2f}'.format)
gr_Age_Group = application[['TARGET','AGE_GROUP','AMT_CREDIT']].groupby(['TARGET','AGE_GROUP'],as_index=False).sum('AMT_CREDIT')
gr_Age_Group

In [ ]:
#pivot the above data so that we can understand the default amount per age group
pivot_Age_Group = gr_Age_Group.pivot(index='AGE_GROUP', columns='TARGET', values='AMT_CREDIT')
pivot_Age_Group.plot(kind='bar', stacked=True)
plt.title("Default Rates per Age Group")
plt.xlabel("Age Group")
plt.ylabel("Default Amount")
plt.show()

- ### Subtask 2.4: create Income_Level field and derive insights based on income level

In [ ]:
#Converting Total Income into categorical data to draw better insights based on income levels
application["INCOME_LEVEL"] = pd.cut(application.AMT_INCOME_TOTAL,[0,100000,200000,300000,9999999999],labels=["LOW","MIDDLE","RICH","ULTRARICH"])
application.INCOME_LEVEL.value_counts(normalize=True).plot.pie(autopct = "%1.0f%%")
plt.title('Customer Composition based on Income Level', fontsize = 12)
plt.ylabel("Income Level", fontsize=10)
plt.show()

In [ ]:
#understanding the default amounts based on income level.
#First groupby the required fields and sumup the total amount of loan per target and income level
pd.set_option('display.float_format', '{:.2f}'.format)
gr_income_level = application[['TARGET','INCOME_LEVEL','AMT_CREDIT']].groupby(['TARGET','INCOME_LEVEL'],as_index=False).sum('AMT_CREDIT')
gr_income_level

In [ ]:
#pivot the above data so that we can understand the default amount per income level
pivot_Income_Level = gr_income_level.pivot(index='INCOME_LEVEL', columns='TARGET', values='AMT_CREDIT')
pivot_Income_Level.plot(kind='bar', stacked=True)
plt.title("Default Rates per Income Level")
plt.xlabel("Income Level")
plt.ylabel("Default Amount")
plt.show()

- ### Subtask 2.5: Analyze DAYS_EMPLOYED field and find outliers. Reduce the dataset to get only legitimate values and then find outliers

In [ ]:
#Analysing and handling outliers in DAYS_EMPLOYED field. This field is supposed to have values less than 0 
#indicating How many days before the application the person started current employment
application.boxplot(column=['DAYS_EMPLOYED'])
plt.show()

In [ ]:
#Since this field is supposed to have values <0, we will ignore any values above 0
pd.set_option('display.float_format', '{:.2f}'.format)
application[application.DAYS_EMPLOYED<=0].describe()

In [ ]:
#creating new dataframe after removing outlier cases for AMT_CREDIT from original dataframe
application_days_employed_correct_data = application[application.DAYS_EMPLOYED<=0]

In [ ]:
#handling outliers in total income field and plotting
application_days_employed_correct_data.boxplot("DAYS_EMPLOYED")
plt.show()

- ### Subtask 2.6: Analyze AMT_CREDIT field and handle outliers

In [ ]:
#handling outliers in loan amount field
application.boxplot(column=['AMT_CREDIT'])
plt.show()

In [ ]:
#finding the correct cutoff based on quantiles
application.AMT_CREDIT.quantile([0.5,0.7,0.9,0.95,0.99,0.995,0.998])

In [ ]:
#Describe to find statistical summary again and observe the change in AMT_CREDIT field. Taking the summary at 95 percentile
pd.set_option('display.float_format', '{:.2f}'.format)
application[application.AMT_CREDIT<1350000].describe()

In [ ]:
#we will lose 16355 rows where AMT_CREDIT is above the 99.8 percentile. This will help in removing the outlier cases
307511- 291156

In [ ]:
#creating new dataframe after removing outlier cases for AMT_CREDIT from original dataframe.Taking the outliers at 95 percentile value
application_loan_outliers = application[application.AMT_CREDIT<=1350000]

In [ ]:
#handling outliers in AMT_CREDIT field and plotting. 
application_loan_outliers.boxplot("AMT_CREDIT")
plt.show()

- ### Subtask 2.7: Analyze AMT_INCOME_TOTAL field and handle outliers

In [ ]:
#handling outliers in loan amount field
application.boxplot(column=['AMT_INCOME_TOTAL'])
plt.show()

In [ ]:
#finding the correct cutoff based on quantiles
application.AMT_INCOME_TOTAL.quantile([0.5,0.7,0.9,0.95,0.99,0.995,0.998])

In [ ]:
#Describe to find statistical summary again and observe the change in AMT_CREDIT field. Taking summary at 99.8 percentile
application[application.AMT_INCOME_TOTAL<337500].describe()

In [ ]:
#we will lose 15825 rows where AMT_INCOME_TOTAL is above the 95 percentile. This will help in removing the outlier cases
307511 -291686

In [ ]:
#creating new dataframe after removing outlier cases for AMT_INCOME_TOTAL from original dataframe.Taking the outliers at 95 percentile value
application_Total_Income_outliers = application[application.AMT_INCOME_TOTAL<=337500]

In [ ]:
#handling outliers in total income field and plotting. 
application_Total_Income_outliers.boxplot("AMT_INCOME_TOTAL")
plt.show()

- ### Subtask 2.8: Understanding Target Variable with respect to other variables<br>

Understanding the data with respect different variable and target variable. This is a very crucial part of data analysis 
and by understanding the various variables available and their potential impact on the target variable, we would be able to
produce a better analysis

- ###### Subtask 2.8.1: GENDER cleanup for XNA values and percentage of defaults

In [ ]:
#checking the distinct gender values and removing XNA values
application.drop(application[application.CODE_GENDER =='XNA'].index, inplace=True)
gr_gender = application[['SK_ID_CURR','TARGET','CODE_GENDER']].groupby(['TARGET','CODE_GENDER']).count()

#pivot the above data so that we can understand the default amount per gender
gr_gender.reset_index().pivot('CODE_GENDER', 'TARGET', 'SK_ID_CURR').plot.bar(stacked=True)
plt.title("Gender Default Rates")
plt.xlabel("Gender")
plt.ylabel("Default %age")

plt.show()

- ###### Subtask 2.8.2: Contract types and the percentage of defaults

In [ ]:
gr_name_cont_type = application[['SK_ID_CURR','TARGET','NAME_CONTRACT_TYPE']].groupby(['TARGET','NAME_CONTRACT_TYPE']).count()

#pivot the above data so that we can understand the default amount per income level
gr_name_cont_type.reset_index().pivot('NAME_CONTRACT_TYPE', 'TARGET', 'SK_ID_CURR').plot.bar(stacked=True)
plt.title("Contract Type Default Rates")
plt.xlabel("Contract Type")
plt.ylabel("Default %age")
plt.show()

- ###### Subtask 2.8.3: Loans granted and education type

In [ ]:
#Finding mean total income based on educationtype. Comparing the loan granted with respect to avg income per incometype.
#Using subplots to compare side-by-side
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.title("Avg Income per Education Type", fontsize=12)
plt.ylabel("Avg Income")
mean_edu = application.groupby('NAME_EDUCATION_TYPE')['AMT_INCOME_TOTAL'].mean()
mean_edu.plot.bar()

plt.subplot(1,2,2)
plt.title("Avg Loan Amount per Education Type", fontsize=12)
plt.ylabel("Avg Loan Amount")
mean_loan_edu = application.groupby('NAME_EDUCATION_TYPE')['AMT_CREDIT'].mean()
mean_loan_edu.plot.bar()

plt.subplots_adjust(wspace=0.5)
plt.show()

- ###### Subtask 2.8.4: Loans granted and occupation type

In [ ]:
#Finding mean total income based on occupationtype. Comparing the loan granted with respect to avg income per occupation
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.title("Avg Income per occupation", fontsize=12)
plt.ylabel("Avg Income")
mean_occu = application.groupby('OCCUPATION_TYPE')['AMT_INCOME_TOTAL'].mean()
mean_occu.plot.bar()

plt.subplot(1,2,2)
plt.title("Avg Loan Amount per occupation", fontsize=12)
plt.ylabel("Avg Loan Amount")
mean_loan_occu = application.groupby('OCCUPATION_TYPE')['AMT_CREDIT'].mean()
mean_loan_occu.plot.bar()

plt.subplots_adjust(wspace=0.5)
plt.show()

- ###### Subtask 2.8.5: Loans granted and income type

In [ ]:
#Finding mean total income based on incometype. Comparing the loan granted with respect to avg income per incometype
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.title("Avg Income per Income Type", fontsize=12)
plt.ylabel("Avg Income")
mean_incometype = application.groupby('NAME_INCOME_TYPE')['AMT_INCOME_TOTAL'].mean()
mean_incometype.plot.bar()

plt.subplot(1,2,2)
plt.title("Avg Loan Amount per Income Type", fontsize=12)
plt.ylabel("Avg Loan Amount")
mean_loan_income = application.groupby('NAME_INCOME_TYPE')['AMT_CREDIT'].mean()
mean_loan_income.plot.bar()

plt.subplots_adjust(wspace=0.5)
plt.show()

- ###### Subtask 2.8.6: other insights

In [ ]:
#the below are a few more dataframes that can be further used to analyse the default rates per variable type. For the purpose of this case study, we are not going to analyse these in more details

gr_owncar = application[['SK_ID_CURR','TARGET','FLAG_OWN_CAR']].groupby(['TARGET','FLAG_OWN_CAR']).count()
gr_ownrealty = application[['SK_ID_CURR','TARGET','FLAG_OWN_REALTY']].groupby(['TARGET','FLAG_OWN_REALTY']).count()
gr_name_type_suite = application[['SK_ID_CURR','TARGET','NAME_TYPE_SUITE']].groupby(['TARGET','NAME_TYPE_SUITE']).count()
gr_family_status = application[['SK_ID_CURR','TARGET','NAME_FAMILY_STATUS']].groupby(['TARGET','NAME_FAMILY_STATUS']).count()
gr_name_housing_type = application[['SK_ID_CURR','TARGET','NAME_HOUSING_TYPE']].groupby(['TARGET','NAME_HOUSING_TYPE']).count()
gr_org_type = application[['SK_ID_CURR','TARGET','ORGANIZATION_TYPE']].groupby(['TARGET','ORGANIZATION_TYPE']).count()

gr_edu_type = application[['SK_ID_CURR','TARGET','NAME_EDUCATION_TYPE']].groupby(['TARGET','NAME_EDUCATION_TYPE']).count()
gr_occ_type = application[['SK_ID_CURR','TARGET','OCCUPATION_TYPE']].groupby(['TARGET','OCCUPATION_TYPE']).count()
gr_income_type = application[['SK_ID_CURR','TARGET','NAME_INCOME_TYPE']].groupby(['TARGET','NAME_INCOME_TYPE']).count()


## Inferences from Data analysis

1) The %age of defaulters are high at around 8% and the major aim of this analysis is to understand the patterns around this and suggest remedies so that %age of defaulters are reduced <br><br>
2) Loans granted to female population are higher than loans granted to male population. Also - The default rate of female population is lesser at 7% compared to male population at 10%<br><br>
3) 85% of the customers are above 30+ years. Default rates are higher for <30 years age group. As people become older, the default rates are getting smaller. The default rates (as a %of loan amount within the age group) for <30 age group is close to 11% where as it is reduced to 5.2% for 60+ agegroup<br><br>
4) Middle Income Group have secured the highest loan amount from the bank. These people have also the highest %age default amount causing losses to the bank<br><br>
5) Cash Loans have higher default at 8.3% compared to Revolving loans at 5.4%<br><br>
6) Highly educated applicants stand a better chance to get higher amount of loan credit<br><br>


- ### Subtask 2.9: Splitting the datasets between defaulters and customers who pay on time<br>

In [ ]:
#picking some relevant columns to find correlation among the customers who pay on time
application_0 = application[['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY',
                             'AMT_GOODS_PRICE','REGION_POPULATION_RELATIVE','CNT_FAM_MEMBERS','REGION_RATING_CLIENT',
                             'EXT_SOURCE_2','DAYS_LAST_PHONE_CHANGE','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE',
                             'OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','AMT_REQ_CREDIT_BUREAU_HOUR',
                             'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',
                             'AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']].loc[application['TARGET'] == 0]

#picking some relevant columns to find correlation among the defaulters
application_1 = application[['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY',
                             'AMT_GOODS_PRICE','REGION_POPULATION_RELATIVE','CNT_FAM_MEMBERS','REGION_RATING_CLIENT',
                             'EXT_SOURCE_2','DAYS_LAST_PHONE_CHANGE','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE',
                             'OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','AMT_REQ_CREDIT_BUREAU_HOUR',
                             'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',
                             'AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']].loc[application['TARGET'] == 1]

In [ ]:
#Finding correlation between different datafields for customers who pay on time
corr0 = application_0.corr()
msk = np.zeros_like(corr0)
msk[np.triu_indices_from(msk)] = True
f, ax = plt.subplots(figsize = (8,5))
with sns.axes_style("white"):
    ax = sns.heatmap(corr0, mask=msk, vmax=.3, square=True)
    plt.show()

In [ ]:
#Finding correlation between different datafields for customers who pay on time
corr1 = application_1.corr()
msk = np.zeros_like(corr1)
msk[np.triu_indices_from(msk)] = True
f, ax = plt.subplots(figsize = (8,5))
with sns.axes_style("white"):
    ax = sns.heatmap(corr1, mask=msk, vmax=.3, square=True)
    plt.show()

- ### Subtask 2.10: Merge application and previous application datasets

In [ ]:
# Read the csv file using 'read_csv'. I have moved the file into my working directory. Using set_option to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
prevapplication = pd.read_csv("previous_application.csv")
prevapplication.head()

In [ ]:
merged_df = application.merge(prevapplication, left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR', how='inner')
merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
#Removing XNA industry to identify clusters of other industries where most applications are made
merged_df_industry = merged_df[['SK_ID_CURR','NAME_SELLER_INDUSTRY','SK_ID_PREV']].loc[merged_df['NAME_SELLER_INDUSTRY'] !="XNA"]
merged_df_industry.head()

In [ ]:
#grouping the industry dataframe
industry_applicants = merged_df_industry[['SK_ID_PREV','NAME_SELLER_INDUSTRY']].groupby(['NAME_SELLER_INDUSTRY'],as_index=False).count()

#plotting a pie to understand where most applications are made
industry_applicants.SK_ID_PREV.plot.pie(autopct = "%1.0f%%")
plt.title('Industry wise Loan Application', fontsize = 12)
plt.ylabel("Industry", fontsize=10)
plt.show()

industry_applicants.sort_values("SK_ID_PREV", ascending=False)


In [ ]:
#inspecting the merged dataset based on the client type and contract statuses. Each of the client types are independent but provides a good relative view of the size of bars
gr_contract_status = merged_df[['SK_ID_PREV','NAME_CONTRACT_STATUS','NAME_CLIENT_TYPE']].groupby(['NAME_CONTRACT_STATUS','NAME_CLIENT_TYPE']).count()

#pivot the above data so that we can understand the default amount per gender
gr_contract_status.reset_index().pivot('NAME_CLIENT_TYPE', 'NAME_CONTRACT_STATUS', 'SK_ID_PREV').plot.bar(stacked=True)
plt.title("Client Wise Loan Status")
plt.xlabel("Contract Status")
plt.ylabel("%age")

plt.show()

In [ ]:
# Checking the 'object' variables of Previous application data(prevapplication). Doing this in a different way than the above stacked charted for practice
# for 'refused', 'approved' & 'canceled' Contract status

refused = prevapplication[prevapplication.NAME_CONTRACT_STATUS=='Refused']
approved = prevapplication[prevapplication.NAME_CONTRACT_STATUS=='Approved']
canceled = prevapplication[prevapplication.NAME_CONTRACT_STATUS=='Canceled']

fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(15,5)) 
chart1 = sns.countplot(ax=ax1,x=refused['PRODUCT_COMBINATION'], data=refused, 
              order= refused['PRODUCT_COMBINATION'].value_counts().index, orient="h")
ax1.set_title("Refused", fontsize=10)
ax1.set_xlabel('%s' %'PRODUCT_COMBINATION')
ax1.set_ylabel("Count of Loans")
chart1.set_xticklabels(chart1.get_xticklabels(), rotation=45, horizontalalignment='right')

chart2= sns.countplot(ax=ax2,x=approved['PRODUCT_COMBINATION'], data=approved, 
              order= approved['PRODUCT_COMBINATION'].value_counts().index,orient="h")
ax2.set_title("Approved", fontsize=10)
ax2.set_xlabel('%s' %'PRODUCT_COMBINATION')
ax2.set_ylabel("Count of Loans")
chart2.set_xticklabels(chart2.get_xticklabels(), rotation=45, horizontalalignment='right')

chart3 = sns.countplot(ax=ax3,x=canceled['PRODUCT_COMBINATION'], data=canceled, 
              order= canceled['PRODUCT_COMBINATION'].value_counts().index,orient="h")
ax3.set_title("Canceled", fontsize=10)
ax3.set_xlabel('%s' %'PRODUCT_COMBINATION')
ax3.set_ylabel("Count of Loans")
chart3.set_xticklabels(chart3.get_xticklabels(), rotation=45, horizontalalignment='right')

plt.show()

## Inferences from Data analysis

1) The merged dataframe has significantly higher row count compared to application dataset indicating that there are multiple previous applications for one current application <br><br>
2) It is very clear from the pie chart that consumer electronics and Connectivity industries are the ones where most of the loans applications are made. Hence, these industries will give most loan business to the bank<br><br>
3) Repeater clients have a good number of canceled and Unused offer loans, so the banks loan marketing departments can follow-up with these customers<br><br>
4) Most of refused loans were for Product combination of 'Cash X-Sell:low'. It is also clear that most of loans were approved for 'POS household with interest'. And most Canceled loans were for 'Cash loans'<br><br>